In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm.notebook import tqdm

In [2]:
def get_words(dataset=3):
    datasets = ['../datasets/names.txt',
                '../datasets/male_names_rus.txt',
                '../datasets/female_names_rus.txt',
                '../datasets/russian_surnames.txt',
                '../datasets/towns.txt']

    filename = datasets[dataset]

    if filename is None:
        print(f'Dataset "{dataset}" not found')
        return []
    
    data = open(filename, 'r').read().lower()

    if dataset == 3:
        data = data\
            .replace('a', 'а')\
            .replace('c', 'с')\
            .replace('k', 'к')\
            .replace('o', 'о')\
            .replace('\x1b', '')\
            .replace('1', '')
    
    return [w.strip() for w in data.splitlines()]

In [39]:
words = get_words(3)
print(f'{len(words)=}')
print(f'{words[:10]=}')

len(words)=251337
words[:10]=['аалферова', 'ааль', 'ааман', 'аамана', 'ааманая', 'ааманий', 'аандреева', 'аарон', 'ааронова', 'аб']


In [4]:
# The most intuitive approach to the Make More problem is to count how many times each character
#   appears on the second place in all pairs of characters (bigrams) we can find in the dataset
# In this example our goal is to combine characters into words which means we should predict the next character
#   potentially given some input characters
# It is a classification task with len(characters) + 1 classes (here and below we use len(characters) + 1 = 27 for brevity)
#   and the output is a vector op probabilites
# This also implies awareness of the start and the end of each word which we achive by adding a special character 0
#   before and after each word

class Counter():
    def __init__(self, words):
        self.chars = ['.'] + sorted(list(set(''.join(words)))) # '.' is going to be used to indicate start and end of words

        self.stoi = {s: i for i, s in enumerate(self.chars)}
        self.itos = {i: s for s, i in self.stoi.items()}

        print(f'total words {len(words)}, total chars {len(self.chars)}, chars{self.chars}')
        
        # Count all possible bigrams
        # count = {}
        # for w in words:
        #     word = '.' + w + '.'
        #     for c1, c2 in zip(word, word[1:]):
        #         bigram = (c1, c2)
        #         count[bigram] = count.get(bigram, 0) + 1
        # sorted(count.items(), key=lambda kv: -kv[1])[:20]

        self.N = torch.zeros((len(self.chars), len(self.chars)), dtype=torch.int32)
        self.P = None
        self.g_prob = None
        self.g = None

        # Bigram occurances Tensor
        #   where rows represent the first character index and columns the last one
        for w in tqdm(words, 'Processing words'):
            word = '.' + w + '.'
            for c1, c2 in zip(word, word[1:]):
                x = self.stoi[c1]
                y = self.stoi[c2]
                self.N[x, y] += 1

    def plot_char_map(self):
        # plt.imshow(N)

        plt.figure(figsize=(16, 16))
        plt.imshow(self.N, cmap='Blues')

        for i in range(len(self.chars)):
            for j in range(len(self.chars)):
                ch_str = self.itos[i] + self.itos[j]
                plt.text(j, i, ch_str, ha='center', va='bottom', color='gray')
                plt.text(j, i, self.N[i, j].item(), ha='center', va='top', color='gray')

        plt.axis('off')
        
    def prob(self, x):
        # The output is a probability distribution vector (sums to 1)
        # We can generate one by deviding each value in a row by the sum of all counts in the row
        # Each row represent all 27 possible labels y for a given input x in bigram (x, y)
        
        # For a single row
        p = self.N[x].float()
        return p / p.sum()
    
    def make_more_prob(self, x=10):
        # Now we can generate random names based on the probability of occurance of each pair of characters
        # To do so we take an input char and choose a random bigram where it appears first with respect
        #   to the corresponding probability distribution vector stored in N
        # 
        # See more here at https://pytorch.org/docs/stable/generated/torch.multinomial.html

        if self.g_prob is None:
            self.g_prob = torch.Generator().manual_seed(2147483647)
        
        for i in range(x):
            name = []
            ix = 0 # 0 -> '.' the start of the word
            while True:
                p = self.prob(ix)
                ix = torch.multinomial(p, num_samples=1, replacement=True, generator=self.g_prob).item()
                name.append(self.itos[ix])
                if ix == 0: # 0 -> '.' the end of the word
                    break

            print(''.join(name))

        return self

    def forward(self):
        # The same thing as in prob() but converting the entire tensor into a probability one

        P = (self.N + 1).float() # Adding +1 for "smoothing" so no pairs get 0 probability for us to never fall into log(0) case (see below)

        # Using `keepdim` is crucial here (see the docs here: https://pytorch.org/docs/stable/generated/torch.sum.html)
        # We need to sum up the values in each row, which means the shape of the result should be (27, 1) - 27 rows 1 value each
        # If we don't keep the dimentions torch will discard this dimension and make it (27) instead of (27, 1).
        # So when it comes to tensor division according to the division rules (27, 27)/(27) is treated like (27, 27)/(1, 27)
        #   which will produce a completely different result from (27, 27)/(27, 1) as it should be
        self.P = P / P.sum(1, keepdim=True)

        # print(f'{self.P[0].sum().item()=}') # should be 1, as it is a sum of probabilities of all pairs starting with char itos[0]

        return self

    def make_more(self, x=10):
        if self.g is None:
            self.g = torch.Generator().manual_seed(2147483647)

        self.forward()

        for i in range(x):
            name = []
            ix = 0
            while True:
                p = self.P[ix]
                ix = torch.multinomial(p, num_samples=1, replacement=True, generator=self.g).item()
                name.append(self.itos[ix])
                if ix == 0:
                    break

            print(''.join(name))
            
        return self

    # here we use words to derive labels and estimate the loss using it
    def loss(self, words):
        # Now as we've learned pair probabilities from the initial dataset of words
        #   we'd like to figure out the quality of our model,
        #   i.e. how good our probabilities (aka weights of the model) are
        # So the loss function we need to use is a product of all probability distribution vectors
        #   of all character pairs aka likelihood
        # For convenience we can take log() of it to leverage the property of log(a*b) = log(a) + log(b)
        # We then also negate the result to match the semantics of "minimization of the loss function",
        #   and also consider the average value as the one we eventually need to minimize the closer to 0 the better

        log_likelihood = 0.0
        pairs_count = 0

        # for w in ['denis']:
        for w in tqdm(words, 'Processing words'):
            chs = '.' + w + '.'
            for ch1, ch2 in zip(chs, chs[1:]):
                ix1 = self.stoi[ch1]
                ix2 = self.stoi[ch2]
                p = self.P[ix1, ix2]
                likelihood = torch.log(p)
                log_likelihood += likelihood
                pairs_count += 1
                # print(f'{ch1}{ch2} {p:.4f} {likelihood:.4f}')

        nll = -log_likelihood
        anll = nll / pairs_count

        print(f'                 log likelihood: {log_likelihood}')
        print(f'        negative log likelihood: {nll}')
        print(f'average negative log likelihood: {anll}')

In [5]:
counter = Counter(words)

total words 1117, total chars 35, chars['.', ' ', '-', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ы', 'ь', 'э', 'ю', 'я', 'ё']


Processing words:   0%|          | 0/1117 [00:00<?, ?it/s]

In [6]:
counter.make_more_prob(10)
print('=' * 50)
counter.make_more(10)
print('=' * 50)
counter.loss(words)

к.
мибубаниноле.
новоляноволо.
бежейск.
амородов.
тай лыбав.
к.
пелещарбибаник.
жнь.
юразновогускивагурстовецымсск.
к.
мибубаниноле.
новоляновья.
грежейск.
амэродов.
тай лыбачель-ёурариб-баник.
жнь.
юразновогу пскагурсткирсг.
бск.
з.


Processing words:   0%|          | 0/1117 [00:00<?, ?it/s]

                 log likelihood: -26081.16796875
        negative log likelihood: 26081.16796875
average negative log likelihood: 2.4912760257720947


In [7]:
# =================================================================
#
#     Now let's get the same results using a Neural Network
#
# =================================================================

In [40]:
class NN():
    def __init__(self, words, lr=50):
        self.chars = ['.'] + sorted(list(set(''.join(words)))) # '.' is going to be used to indicate start and end of words

        self.stoi = {s: i for i, s in enumerate(self.chars)}
        self.itos = {i: s for s, i in self.stoi.items()}

        print(f'total words {len(words)}, total chars {len(self.chars)}, chars{self.chars}')
        
        # Starting with the code from above iterating over all bigrams,
        #   but instead of counting let's form a training set of bigrams (x, y)
        #   using 'xs' as inputs and 'ys' as labels

        xs, ys = [], []

        # for w in words[:1]: # let's first run this for a single word
        for w in tqdm(words, 'Processing words'):
            word = '.' + w + '.'
            for c1, c2 in zip(word, word[1:]):
                x = self.stoi[c1]
                y = self.stoi[c2]
                xs.append(x)
                ys.append(y)

        self.xs = torch.tensor(xs) # it is important that we should have integers here and below
        self.ys = torch.tensor(ys)

        print(f'Total bigrams processed {len(self.xs)}')

        self.g_make_more = torch.Generator().manual_seed(2147483647)
        self.g_weights = torch.Generator().manual_seed(2147483647)
        # self.g_weights = torch.Generator()
        
        # Now we can create a random weight matrix and then multiply by input vector (see x_enc below)
        # In order to have a (1, 27) probability distribution vector for each x the weight matrix should be (27, 27)
        # Such matrix represents a single linear layer of our NN and
        # the forward pass is going to be XW + B (in our case B = 0)
        self.W = torch.randn((len(self.chars), len(self.chars)), # 27 neurons NN, each neuron has 27 inputs
                             generator=self.g_weights,
                             requires_grad=True)

        self.loss = None
        self.lr = lr
        self.P = None

    # Forward pass
    def forward(self):

        # for xs (the inputs) instead of just ints (char codes) it is more convenient
        # (in terms of vector operations) to use one-hot transformed values instead

        x_enc = F.one_hot(self.xs, num_classes=len(self.chars)).float() # the vectors should be float to fit into NN

        # print(self.x_enc)
        # print(f'{self.x_enc.shape=}')
        # plt.imshow(self.x_enc)

        # The output is the updated probability matrix (tensor)
        # To construct a probability matrix we are going to use exp() to get positive floats as
        #   we consider original values to be log(count), so count = exp(log(count)),
        #   and then again we normalize the counts to get the probabilities as we did above

        logits = x_enc @ self.W # '@' is matrix multiplication in torch; log(count) pridiction we get here

        # The next two lines together are called 'Softmax' and turn random positive and negative numbers
        #   into a probability distribution vector, all the values of which sum to 1
        count = logits.exp() # exp(log(count)) = count
        self.P = count / count.sum(1, keepdim=True) # mind keeping the dimentions explicitly as explained above

        # Calculating loss manually

        # nlls = torch.zeros(5) # 5 = len(self.xs) here
        # for i in range(5):
        #     xi = self.xs[i].item() # input character
        #     yi = self.ys[i].item() # label
        #     p = self.P[i, yi]
        #     ll = torch.log(p)
        #     nll = -ll
        #     nlls[i] = nll
        #     print(f'{i+1}. Input input {itos[xi]} target {itos[yi]} guess {P[i]}, '+
        #           f'probability assigned {p.item()} log likelihood {ll.item()} nll {nll.item()}')
        #
        # print(f'Average nll (loss) {nlls.mean().item()}')

        # Calculating loss
        self.loss = -self.P[torch.arange(len(self.xs)), self.ys].log().mean()
        return self
    
    # Backward pass
    def backward(self):
        self.W.grad = None # reset all gradients from the previous pass
        self.loss.backward()
        self.W.data += -self.lr * self.W.grad
        return self
        
    def train(self):
        self.forward()
        self.backward()
        return self
    
    def learn(self, x=100):
        for i in tqdm(range(x), 'Learning progress'):
            self.train()

        return self
    
    def make_more(self, x=10):
        self.forward()
        for i in range(x):
            word = []
            ix = 0 # stoi['.']
            while True:
                x_enc = F.one_hot(torch.tensor([ix]), num_classes=len(self.chars)).float()
                logits = x_enc @ self.W
                count = logits.exp()
                p = count / count.sum(1, keepdim=True)
                ix = torch.multinomial(p, num_samples=1, replacement=True, generator=self.g_make_more).item()
                word.append(self.itos[ix])
                if ix == 0:
                    break

            print(''.join(word))

In [41]:
nn = NN(words)

total words 251337, total chars 35, chars['.', '-', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё']


Processing words:   0%|          | 0/251337 [00:00<?, ?it/s]

Total bigrams processed 2330339


In [43]:
nn.learn(100)
print(f'{nn.loss=}')

Learning progress:   0%|          | 0/100 [00:00<?, ?it/s]

nn.loss=tensor(2.4189, grad_fn=<NegBackward0>)


In [37]:
nn.make_more(20)

к.
мибубаниноле.
новоляноволо.
бежейск.
амэродов.
тай лыбав.
к.
пелещарб-атник.
жнь.
юразновогускивагурстовецговск.
з.
нглюрск.
овоско.
амав.
цк.
няньспызовецый.
скотрк.
цгул.
ся нрлеранаернстеегбимрганснсана.
ов.


In [38]:
print(f'{nn.P[torch.arange(len(nn.xs)), nn.ys].shape=}')
print(f'{nn.P[:,nn.ys].shape=}')

nn.P[torch.arange(len(nn.xs)), nn.ys].shape=torch.Size([10469])
nn.P[:,nn.ys].shape=torch.Size([10469, 10469])
